In [5]:
#@markdown ## ⚠️ PLEASE NOTE:
#@markdown This colab runs best using a GPU runtime.  From the Colab menu, choose Runtime > Change Runtime Type, then select **'GPU'** in the dropdown.

import functools
import jax
import os

from datetime import datetime
from jax import numpy as jp
import matplotlib.pyplot as plt

from IPython.display import HTML, clear_output

try:
  import brax
except ImportError:
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

try:
  import wandb
except:
  !pip install wandb
  clear_output()
  import wandb
  
import flax
from brax import envs
from brax.io import model
from brax.io import json
from brax.io import html
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import train as sac

if 'COLAB_TPU_ADDR' in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

In [6]:
train_config = {
        "num_timesteps":50000000,
        "num_evals":10,
        "reward_scaling":0.1,
        "episode_length":1000,
        "normalize_observations":True,
        "action_repeat":1,
        "unroll_length":10,
        "num_minibatches":32,
        "num_updates_per_batch":8,
        "discounting":0.97,
        "learning_rate":3e-4,
        "entropy_cost":1e-3,
        "num_envs":2048,
        "batch_size":1024,
        "seed": 0xdeadbeef,
    }

In [ ]:
#!/usr/bin/env python3

import functools
import os
import sys
from pathlib import Path
import pickle
import logging

# import gin
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=16'
import jax

from datetime import datetime
from jax import numpy as jp
import brax

import flax
from brax import envs
from brax.io import model
from brax.io import json
from brax.io import html
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import train as sac
from brax.training.agents.apg import train as apg
import wandb


wandb_init = wandb.init
cpu_device = jax.devices("cpu")[0]

logger = logging.getLogger('my_logger')

logging.basicConfig(
    level=logging.INFO, # allow DEBUG level messages to pass through the logger
    force=True
)

logging.info("Starting")

def progress(num_steps, metrics):
    logging.info(f"{num_steps=}")
    logging.info(f"{metrics=}")

    for key, value in metrics.items():
      if isinstance(value, jp.ndarray):
        metrics[key] = jax.device_put(value, cpu_device)
    
    wandb.log(metrics, step=num_steps)


# @gin.configurable
def training_main(
        env_name: str,
        backend: str,
        experiment_name: str,
        log_store_dir: str,
        train_config: dict,
):
    exp_dir = f"{log_store_dir}/{experiment_name}/{env_name}"
    Path(exp_dir).mkdir(parents=True, exist_ok=True)

    seed = train_config.get("seed")
    env = envs.get_environment(env_name=env_name,
                               backend=backend)
    train_fn = functools.partial(
        ppo.train,
        **train_config,
    )

    # no way to save intermediate params, only at very end
    make_inference_fn, params, _ = train_fn(
        environment=env,
        progress_fn=progress,
    )
    model.save_params(f"{exp_dir}/params.pickle", params)
    wandb.save(exp_dir)
    return env_name, backend, make_inference_fn, params


def eval_main(
    env_name,
    backend,
    make_inference_fn,
    params,
):
    env = envs.get_environment(env_name=env_name, backend=backend)
    inference_fn = make_inference_fn(params)
    jit_env_reset = jax.jit(env.reset)
    jit_env_step = jax.jit(env.step)
    jit_inference_fn = jax.jit(inference_fn)
    for rollout_idx in range(10):
        logging.info(f"{rollout_idx=}")
        rollout = []
        rng = jax.random.PRNGKey(seed=rollout_idx)
        state = jit_env_reset(rng=rng)
        act_rng, rng = jax.random.split(rng)
        done = False
        while not done:
            act, _ = jit_inference_fn(state.obs, act_rng)
            state = jit_env_step(state, act)
            rollout.append(state.pipeline_state)
            done = state.done.any()
        logging.info(f"{rollout_idx=} {len(rollout)=}")
        # TODO bzs save render to json
        wandb.Html(html.render(env.sys.replace(dt=env.dt), rollout))



wandb_init(
  project="timewise",
  entity = "e1even",
  mode="online",
  name="humanoid_ppo_trial",
  config = train_config
)
try:
    env_name, backend, make_inference_fn, params = training_main(
      env_name = "humanoid",
      backend =  "generalized",
      experiment_name = "humanoid",
      log_store_dir = "humanoid",
      train_config = train_config)
    eval_main(env_name, backend, make_inference_fn, params)
except KeyboardInterrupt:
    logging.warning("user quit")
finally:
    wandb.finish()

INFO:root:Starting
wandb: Currently logged in as: onatv (e1even). Use `wandb login --relogin` to force relogin


INFO:absl:Device count: 1, process count: 1 (id 0), local device count: 1, devices to be used count: 1
